## import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import seaborn as sns; sns.set()
import lightgbm as lgb
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import re
from bs4 import BeautifulSoup
from urllib import request
from urllib.request import urlopen
import time
from sklearn.preprocessing import StandardScaler

## class・関数 定義

In [5]:
class prediction:
    
    def __init__(self):
        self.get_jcd()
        self.my_list = []
        self.get_predict_day()
        self.get_rno()
        self.read_csv()
        self.process()
        self.gbm()
        self.predict()
        
    def get_predict_day(self):
        self.predict_day = input('予測したい日をyyyymmdd型で入力:')
        
    def get_jcd(self):
        #2桁で入力やで
        self.jcd = input('予測したい場を2桁で入力:')
        
    def get_rno(self):
        self.rno = input('予測したいラウンドを入力:')
    
    def read_csv(self):
        self.df = pd.read_csv('/volume_dir/boatrace/merged_df.csv')
        self.odds = pd.read_csv('/volume_dir/boatrace/odds_csv/odds.csv')
    
    def process(self):
        # ここで前処理を行う
    def gbm(self):
        # ここでGBMを行う
        pass
    
    def collect_data(self):
        rno
        url='https://www.boatrace.jp/owpc/pc/race/racelist?rno='+str(rno)+\
            '&jcd='+str(self.jcd).zfill(2)+'&hd='+str(self.predict_day)
        time.sleep(1)
        response=request.urlopen(url)
        soup=BeautifulSoup(response,'html.parser')
        response.close()
        data = []
        for i in range(0,6):
            table = soup.find_all('tbody',class_='is-fs12')[i].get_text()
            # 番号を抽出する
            number = re.search(r'\d+', table).group()
            # 名前を抽出する
            name = re.search(r'\n\n(.+)\n\n(.+)\n', table).group(1).strip().replace('\u3000','')
            # 場所を抽出する
            location = re.search(r'\n\n(.+)\n\n(.+)\n', table).group(2).strip().replace('/','')
            # 年齢と体重を抽出する
            age, weight = re.search(r'(\d+歳)/(\d+\.\d+kg)', table).groups()
            # 数字のリストを抽出する
            numbers = re.findall(r'(\d+\.\d+)', table)
            a_numbers = re.findall(r'(\d+)',table)
            location = location[:2]
            age = age.replace('歳','')
            weight = weight.replace('kg','')
            motor = a_numbers[22]
            boat= a_numbers[27]
            
            data.append([number,name,location,age,*numbers,motor,boat])
        df_1 = pd.DataFrame(data,columns=['position','name','area','age','weight','st_time_ave','win',\
                'win_p_2','win_p','a_win','a_win_p_2','a_win_p','motor_p_2','motor_p','boat_p_2','boat_p','motor','boat'])
        url = 'https://www.boatrace.jp/owpc/pc/race/beforeinfo?rno='+str(rno)+\
                '&jcd='+str(jcd).zfill(2)+'&hd='+str(predict_day)
        time.sleep(1)
        response = request.urlopen(url)
        soup = BeautifulSoup(response,'html.parser')
        response.close()
        data_2 = []
        for i in range(0,6):
            table = soup.find_all('tbody',class_='is-fs12')[i].get_text()
            table = table.replace('\n\n','').replace('\u3000','').replace('\n',',')
            table_data = table.split(',')
            position_name = table_data[0]
            name = position_name[1:]
            display = table_data[2]
            table2 = soup.find_all('tbody',class_='is-p10-0')[0].get_text()
            table2 = table2.replace('\n\n\n','').replace('\n\n',',').replace('\n',',')
            table2_data = table2.split(',')
            st_times = {'st_time_1':table2_data[2],'st_time_2':table2_data[4],'st_time_3':table2_data[6],\
                'st_time_4':table2_data[8],'st_time_5':table2_data[10],'st_time_6':table2_data[12]}
            table3 = soup.find('div',class_='weather1_body').get_text()
            table3 = table3.replace('\n\n\n\n','').replace('\n',',')
            table3_data = table3.split(',')
            weather = table3_data[3]
            wind_strong = table3_data[5].replace('m水温','')
            wave = table3_data[8].replace('cm','')
            data_2.append([name,display,st_times['st_time_{}'.format(i+1)],weather,wind_strong,wave])
        df_2 = pd.DataFrame(data_2,columns=['name','display','st_time','weather','wind_strong','wave'])
        # dfの前処理
        df = pd.merge(df_1,df_2,on=['name'])
        int_columns = ['age','weight','position','st_time_ave','win','win_p_2', 'win_p', 'a_win', 'a_win_p_2', 'a_win_p', 'motor_p_2',\
        'motor_p', 'boat_p_2', 'boat_p', 'motor', 'boat', 'display', 'st_time',\
        'wind_strong', 'wave']
        for i in int_columns:
            df[i] = df[i].astype(float)
        std_columns = ['win','win_p_2', 'win_p', 'a_win', 'a_win_p_2', 'a_win_p', 'motor_p_2',\
            'motor_p', 'boat_p_2', 'boat_p']
        for i in std_columns:
            win_mean=df[i].mean()
            win_std=df[i].std()
            if win_std==0: #もしdf_stdが同じ値だとwin_stdが0になってしまう
                df[i]=50.0
            else:
                df[i] = df_std.apply(lambda x : ((x - win_mean)*10/win_std+50))
        
        df['race_date'] = predict_day
        
            

IndentationError: expected an indented block (1797522460.py, line 29)